### Importowanie pakietów oraz zdefiniowanie funkcji obrotu

In [1]:
from package import xaxisMatrixrotation, yaxisMatrixrotation,zaxisMatrixrotation, rotationMatrixZYX,NormalizeVector,multiQuaternion,normOfQuaternion
import math
import imageio
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [2]:
# Obrot punktu metoda macierzową
def RotationbyMatrix(rotation_matix, point):
    return np.matmul(rotation_matix, point)

### Funkcja generująca obrazek obruconego bączka z użyciem macierzy obrotu

In [3]:
def spin3dMatrixRotation(speed:float,euler_angle:list,time:int):
    #main_points
    base_point=[0,0,0]
    end_point=[0,0,3]
    h1=1
    r1=1
    h2=1+1/3 
    r2=1/2
    round_1 = np.linspace(0, 2*math.pi, 10**3)
    first_cycle:list  = []
    second_cycle:list = []
    color_change_frequency:int=100
    k=0
    for i in round_1:
        first_cycle.append([r1*math.cos(i),r1*math.sin(i),h1])
        second_cycle.append([r2*math.cos(i),r2*math.sin(i),h2])
    #####
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    #ax.axis('off')

    z_axis = np.linspace(0, 4, 100)
    xy_axis = np.linspace(-4, 4, 100)
    z_0=np.linspace(0, 0, 100)
    end_point_plot = np.linspace(10, 10, 200)
    ax.plot(xy_axis, z_0, z_0,color = 'black', linewidth=1,alpha=0)
    ax.plot(z_0, z_0,z_axis,color = 'black', linewidth=1,alpha=0)
    ax.plot(z_0, xy_axis, z_0,color = 'black', linewidth=1,alpha=0)
    ax.view_init(5,30,0)

    ###
    first_cycle_rotation:list = [] 
    second_cycle_rotation:list = [] 
    RotMatZaxis = zaxisMatrixrotation(speed)
    RotationMatrixEuler = rotationMatrixZYX(euler_angle[0],euler_angle[1],euler_angle[2])
    i=0
    while i < len(first_cycle):
        ##
        element1=RotationbyMatrix(RotMatZaxis,first_cycle[i])
        element2=RotationbyMatrix(RotMatZaxis,second_cycle[i])
        # 
        first_cycle_rotation.append(RotationbyMatrix(RotationMatrixEuler,element1))
        second_cycle_rotation.append(RotationbyMatrix(RotationMatrixEuler,element2))
        i=i+1
    first_cycle=first_cycle_rotation
    second_cycle=second_cycle_rotation
    ##
    end_point  = RotationbyMatrix(RotMatZaxis,end_point)
    ##
    end_point  =  RotationbyMatrix(RotationMatrixEuler,end_point)
 
    i=0
    lista_test1=[]
    while i < len(first_cycle)-1:
        #1level
        xr1=[base_point[0],first_cycle[i][0],first_cycle[i+1][0]]
        yr1=[base_point[1],first_cycle[i][1],first_cycle[i+1][1]]
        zr1=[base_point[2],first_cycle[i+1][2],first_cycle[i+1][2]]
        verts1 = [list(zip(xr1,yr1,zr1))]
        #2level
        xr2=[first_cycle[i][0],second_cycle[i][0],second_cycle[i+1][0],first_cycle[i+1][0]]
        yr2=[first_cycle[i][1],second_cycle[i][1],second_cycle[i+1][1],first_cycle[i+1][1]]
        zr2=[first_cycle[i][2],second_cycle[i+1][2],second_cycle[i+1][2],first_cycle[i+1][2]]
        verts2 = [list(zip(xr2,yr2,zr2))]
        #3level
        xr3=[end_point[0],second_cycle[i][0],second_cycle[i+1][0]]
        yr3=[end_point[1],second_cycle[i][1],second_cycle[i+1][1]]
        zr3=[end_point[2],second_cycle[i+1][2],second_cycle[i+1][2]]
        verts3 = [list(zip(xr3,yr3,zr3))]
        ##
        if_list = [i % color_change_frequency, round(i/color_change_frequency)]
        lista_test1.append(if_list)
        if if_list[1] % 2 == 0:
            ax.add_collection3d(Poly3DCollection(verts1,facecolors='yellow', linewidths=1))
            ax.add_collection3d(Poly3DCollection(verts2,facecolors='black', linewidths=1))
            ax.add_collection3d(Poly3DCollection(verts3,facecolors='yellow', linewidths=1))
        if if_list[1] % 2 == 1:
            ax.add_collection3d(Poly3DCollection(verts1,facecolors='black', linewidths=1))
            ax.add_collection3d(Poly3DCollection(verts2,facecolors='yellow', linewidths=1))
            ax.add_collection3d(Poly3DCollection(verts3,facecolors='black', linewidths=1))
        i=i+1


    #plt.title('Kąt obrotu (' + eulerz + ', '+ eulery + ', '+ eulerx+')')
    
    plt.savefig(f'./AnimationMatrix/Animation{time}.png',
                dpi=500, 
                transparent = False,  
                facecolor = 'white')
 
    plt.close()

### Generowanie animacji

In [ ]:
size       = 500
time       = 0
axisAngle  = np.linspace(0,32*math.pi, size)
radius     = np.linspace(0,1/2, size)
speedValue = np.linspace(16*(math.pi),math.pi/8, size)

while time < size:
    spin3dMatrixRotation(speedValue[time],[0,radius[time]*math.sin(axisAngle[time]),radius[time]*math.cos(axisAngle[time])],time)
    time = time +1

In [ ]:
frames = []
t = 0
while t<500:
    image = imageio.v2.imread(f'./AnimationMatrixTest/Animation{t}.png')
    frames.append(image)
    t=t+1
imageio.mimwrite('./AnimationMatrixEuler.gif', # output gif
                frames,
                duration=20,
                loop = 0)         # optional: frames per second

### Optymalizacja

In [ ]:
def spin3dMatrixRotationOptimalized(euler_angle:list,time:int):
    #main_points
    base_point=[0,0,0]
    end_point=[0,0,3]
    h1=1
    r1=1
    h2=1+1/3 
    r2=1/2
    round_1 = np.linspace(0, 2*math.pi, 10**3)
    first_cycle:list  = []
    second_cycle:list = []
    color_change_frequency:int=100
    k=0
    for i in round_1:
        first_cycle.append([r1*math.cos(i),r1*math.sin(i),h1])
        second_cycle.append([r2*math.cos(i),r2*math.sin(i),h2])
    #####
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    #ax.axis('off')

    z_axis = np.linspace(0, 4, 100)
    xy_axis = np.linspace(-4, 4, 100)
    z_0=np.linspace(0, 0, 100)
    end_point_plot = np.linspace(10, 10, 200)
    ax.plot(xy_axis, z_0, z_0,color = 'black', linewidth=1,alpha=0)
    ax.plot(z_0, z_0,z_axis,color = 'black', linewidth=1,alpha=0)
    ax.plot(z_0, xy_axis, z_0,color = 'black', linewidth=1,alpha=0)
    ax.view_init(5,30,0)

    ###
    first_cycle_rotation:list = [] 
    second_cycle_rotation:list = [] 
    RotationMatrixEuler = rotationMatrixZYX(euler_angle[0],euler_angle[1],euler_angle[2])
    i=0
    while i < len(first_cycle):
        ##
        # 
        first_cycle_rotation.append(RotationbyMatrix(RotationMatrixEuler,first_cycle[i]))
        second_cycle_rotation.append(RotationbyMatrix(RotationMatrixEuler,second_cycle[i]))
        i=i+1
    first_cycle=first_cycle_rotation
    second_cycle=second_cycle_rotation
    ##
    ##
    end_point  =  RotationbyMatrix(RotationMatrixEuler,end_point)
 
    i=0
    lista_test1=[]
    while i < len(first_cycle)-1:
        #1level
        xr1=[base_point[0],first_cycle[i][0],first_cycle[i+1][0]]
        yr1=[base_point[1],first_cycle[i][1],first_cycle[i+1][1]]
        zr1=[base_point[2],first_cycle[i+1][2],first_cycle[i+1][2]]
        verts1 = [list(zip(xr1,yr1,zr1))]
        #2level
        xr2=[first_cycle[i][0],second_cycle[i][0],second_cycle[i+1][0],first_cycle[i+1][0]]
        yr2=[first_cycle[i][1],second_cycle[i][1],second_cycle[i+1][1],first_cycle[i+1][1]]
        zr2=[first_cycle[i][2],second_cycle[i+1][2],second_cycle[i+1][2],first_cycle[i+1][2]]
        verts2 = [list(zip(xr2,yr2,zr2))]
        #3level
        xr3=[end_point[0],second_cycle[i][0],second_cycle[i+1][0]]
        yr3=[end_point[1],second_cycle[i][1],second_cycle[i+1][1]]
        zr3=[end_point[2],second_cycle[i+1][2],second_cycle[i+1][2]]
        verts3 = [list(zip(xr3,yr3,zr3))]
        ##
        if_list = [i % color_change_frequency, round(i/color_change_frequency)]
        lista_test1.append(if_list)
        if if_list[1] % 2 == 0:
            ax.add_collection3d(Poly3DCollection(verts1,facecolors='yellow', linewidths=1))
            ax.add_collection3d(Poly3DCollection(verts2,facecolors='black', linewidths=1))
            ax.add_collection3d(Poly3DCollection(verts3,facecolors='yellow', linewidths=1))
        if if_list[1] % 2 == 1:
            ax.add_collection3d(Poly3DCollection(verts1,facecolors='black', linewidths=1))
            ax.add_collection3d(Poly3DCollection(verts2,facecolors='yellow', linewidths=1))
            ax.add_collection3d(Poly3DCollection(verts3,facecolors='black', linewidths=1))
        i=i+1


    #plt.title('Kąt obrotu (' + eulerz + ', '+ eulery + ', '+ eulerx+')')
    
    plt.savefig(f'./AnimationMatrixTestOpti/Animation{time}.png',
                dpi=500, 
                transparent = False,  
                facecolor = 'white')
 
    plt.close()

In [ ]:
size       = 500
time       = 0
axisAngle  = np.linspace(0,32*math.pi, size)
radius     = np.linspace(0,1/2, size)
speedValue = np.linspace(2*(math.pi),math.pi/8, size)

#print(RotationbyMatrix(rotationMatrixFromQuaternionrotation(math.pi/3,[radius[time]*math.cos(axisAngle[time]),radius[time]*math.sin(axisAngle[time]),(1-radius[time])**(1/2)]),[0,0,3]))
#while time<size:
    #print(RotationbyMatrix(rotationMatrixFromQuaternionrotation(math.pi/3,[radius[time]*math.cos(axisAngle[time]),radius[time]*math.sin(axisAngle[time]),(1-radius[time])**(1/2)]),[0,0,3]))
    #spin3dMatrixRotation(speedValue[time],math.pi/2,[radius[time]*math.cos(axisAngle[time]),radius[time]*math.sin(axisAngle[time]),(1-radius[time])**(1/2)],time)
#    time=time+1
while time < size:
    spin3dMatrixRotationOptimalized([speedValue[time],radius[time]*math.sin(axisAngle[time]),radius[time]*math.cos(axisAngle[time])],time)
    time = time +1

In [ ]:
frames = []
t = 0
while t<500:
    image = imageio.v2.imread(f'./AnimationMatrixTestOpti/Animation{t}.png')
    frames.append(image)
    t=t+1
imageio.mimwrite('./AnimationMatrixEulerOpti.gif', # output gif
                frames,
                duration=20,
                loop = 0)         # optional: frames per second

### Obrót macierzowy z użciem kwaternionów

In [79]:
def rotationMatrixFromQuaternionrotation(scalar: float, rotation_axis: list):
    qr = scalar
    qi = rotation_axis[0]
    qj = rotation_axis[1]
    qk = rotation_axis[2]
    s = normOfQuaternion(scalar,rotation_axis)
   # print(a,b,c,d)
    RotationMatrix = np.array(
        [
            [
                1-2*(qj**2+qk**2),
                2 *s**(-2)*(qi*qj - qk*qr) ,
                2 *s**(-2)* (qi * qk + qj * qr),
            ],
            [
                2*s**(-2) * (qi * qj + qk * qr),
                1 -2*(qi**2 + qk**2),
                2 *s**(-2)* (qj*qk - qi*qr),
            ],
            [
                2*s**(-2) * (qi*qk - qj*qr),
                2 *s**(-2)* (qj*qk + qi*qr),
                1-2*(qi**2+qj**2),
            ],
        ]
    )
    return RotationMatrix

In [80]:
rotationMatrixFromQuaternionrotation(math.pi/3,[1,3,1])

array([[-19.        ,   0.0266907 ,   0.05660687],
       [  0.05531668,  -3.        ,   0.0266907 ],
       [ -0.02927107,   0.05531668, -19.        ]])

In [84]:
def spin3dMatrixRotationQuaternion(speed:float,angle:float,axis_rotation:list,time:int):
    #main_points
    base_point=[0,0,0]
    end_point=[0,0,3]
    h1=1
    r1=1
    h2=1+1/3 
    r2=1/2
    round_1 = np.linspace(0, 2*math.pi, 10**3)
    first_cycle:list  = []
    second_cycle:list = []
    color_change_frequency:int=100
    k=0
    for i in round_1:
        first_cycle.append([r1*math.cos(i),r1*math.sin(i),h1])
        second_cycle.append([r2*math.cos(i),r2*math.sin(i),h2])
    #####
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    #ax.axis('off')

    z_axis = np.linspace(0, 4, 100)
    xy_axis = np.linspace(-4, 4, 100)
    z_0=np.linspace(0, 0, 100)
    end_point_plot = np.linspace(10, 10, 200)
    ax.plot(xy_axis, z_0, z_0,color = 'black', linewidth=1,alpha=0)
    ax.plot(z_0, z_0,z_axis,color = 'black', linewidth=1,alpha=0)
    ax.plot(z_0, xy_axis, z_0,color = 'black', linewidth=1,alpha=0)
    ax.view_init(5,30,0)

    ###
    first_cycle_rotation:list = [] 
    second_cycle_rotation:list = [] 
    #RotMatZaxis = zaxisMatrixrotation(speed)
    q1_scalar = math.cos(speed/2)
    q1_vector = [0,0,math.sin(speed/2)]
    q2_scalar = math.cos(angle/2)
    q2_vector = [math.sin(angle/2)*axis_rotation[0],math.sin(angle/2)*axis_rotation[1],math.sin(angle/2)*axis_rotation[2]]
    q1=multiQuaternion(q2_scalar,q2_vector,q1_scalar,q1_vector)
    rotationMatrixFromQuaternion = rotationMatrixFromQuaternionrotation(q1[0],q1[1])
    #print(np.linalg.det(rotationMatrixFromQuaternion))
    #print(normOfQuaternion(q1[0],q1[1]))
    i=0
    while i < len(first_cycle):
        ##
        # first_cycle_rotation.append(RotationbyMatrix(RotMatZaxis,first_cycle[i]))
        # second_cycle_rotation.append(RotationbyMatrix(RotMatZaxis,second_cycle[i]))
        # 
        first_cycle_rotation.append(RotationbyMatrix(rotationMatrixFromQuaternion,first_cycle[i]))
        second_cycle_rotation.append(RotationbyMatrix(rotationMatrixFromQuaternion,second_cycle[i]))
        i=i+1
    first_cycle=first_cycle_rotation
    second_cycle=second_cycle_rotation

    ##
    end_point  =  RotationbyMatrix(rotationMatrixFromQuaternion,end_point)
 
    i=0
    lista_test1=[]
    while i < len(first_cycle)-1:
        #1level
        xr1=[base_point[0],first_cycle[i][0],first_cycle[i+1][0]]
        yr1=[base_point[1],first_cycle[i][1],first_cycle[i+1][1]]
        zr1=[base_point[2],first_cycle[i+1][2],first_cycle[i+1][2]]
        verts1 = [list(zip(xr1,yr1,zr1))]
        #2level
        xr2=[first_cycle[i][0],second_cycle[i][0],second_cycle[i+1][0],first_cycle[i+1][0]]
        yr2=[first_cycle[i][1],second_cycle[i][1],second_cycle[i+1][1],first_cycle[i+1][1]]
        zr2=[first_cycle[i][2],second_cycle[i+1][2],second_cycle[i+1][2],first_cycle[i+1][2]]
        verts2 = [list(zip(xr2,yr2,zr2))]
        #3level
        xr3=[end_point[0],second_cycle[i][0],second_cycle[i+1][0]]
        yr3=[end_point[1],second_cycle[i][1],second_cycle[i+1][1]]
        zr3=[end_point[2],second_cycle[i+1][2],second_cycle[i+1][2]]
        verts3 = [list(zip(xr3,yr3,zr3))]
        ##
        if_list = [i % color_change_frequency, round(i/color_change_frequency)]
        lista_test1.append(if_list)
        if if_list[1] % 2 == 0:
            ax.add_collection3d(Poly3DCollection(verts1,facecolors='yellow', linewidths=1))
            ax.add_collection3d(Poly3DCollection(verts2,facecolors='black', linewidths=1))
            ax.add_collection3d(Poly3DCollection(verts3,facecolors='yellow', linewidths=1))
        if if_list[1] % 2 == 1:
            ax.add_collection3d(Poly3DCollection(verts1,facecolors='black', linewidths=1))
            ax.add_collection3d(Poly3DCollection(verts2,facecolors='yellow', linewidths=1))
            ax.add_collection3d(Poly3DCollection(verts3,facecolors='black', linewidths=1))
        i=i+1


    #plt.title('Kąt obrotu (' + eulerz + ', '+ eulery + ', '+ eulerx+')')
    
    plt.savefig(f'./AnimationMatrixTest/Animation{time}.png',
                dpi=500, 
                transparent = False,  
                facecolor = 'white')
 
    plt.close()

In [85]:
size       = 500
time       = 0
axisAngle  = np.linspace(0,32*math.pi, size)
radius     = np.linspace(0,1/4, size)
speedValue = np.linspace(16*(math.pi),math.pi/8, size)

#print(RotationbyMatrix(rotationMatrixFromQuaternionrotation(math.pi/3,[radius[time]*math.cos(axisAngle[time]),radius[time]*math.sin(axisAngle[time]),(1-radius[time])**(1/2)]),[0,0,3]))
while time<size:
    #print(RotationbyMatrix(rotationMatrixFromQuaternionrotation(math.pi/3,[radius[time]*math.cos(axisAngle[time]),radius[time]*math.sin(axisAngle[time]),(1-radius[time])**(1/2)]),[0,0,3]))
    spin3dMatrixRotationQuaternion(speedValue[time],math.pi/2,[radius[time]*math.cos(axisAngle[time]),radius[time]*math.sin(axisAngle[time]),(1-radius[time]**2)**(1/2)],time)
    time=time+1

In [4]:
t = 0
frames = []
while t<500:
    image = imageio.v2.imread(f'./AnimationMatrixTest/Animation{t}.png')
    frames.append(image)
    t=t+1
imageio.mimwrite('./AnimationMatrixQuaternion1.gif', # output gif
                frames,
                duration=20,
                loop = 0)         # optional: frames per second

[12889, [5388, 8537, 8150]]